In [1]:
import arcpy

https://pro.arcgis.com/en/pro-app/arcpy/mapping/mapseries-class.htm

https://pro.arcgis.com/en/pro-app/help/layouts/map-series.htm



In [6]:
project_path = r".\Beginner_Week_3.aprx"
project = arcpy.mp.ArcGISProject(project_path)

layouts = project.listLayouts()

In [7]:
for layout in layouts:
    if not layout.mapSeries is None:
        print(layout.mapSeries.enabled)

True


In [8]:
layout

In [11]:
for layer in layout.mapSeries.mapFrame.map.listLayers():
    print(layer.name)

Highways_Intersect
Counties
Topographic


In [31]:
layout.listElements()

In [14]:
roads_layer = layout.mapSeries.mapFrame.map.listLayers()[0]

In [25]:
county = 'Marin County'

In [17]:
ms = layout.mapSeries

In [26]:
ms.currentPageNumber =  ms.getPageNumberFromName(county)

In [27]:
ms.currentPageNumber

12

In [35]:
title = layout.listElements()[0]
title.text = county

In [28]:
roads_layer.definitionQuery = "NAMELSAD = '{}'".format(county)

In [36]:
ms.exportToPDF('./{}.pdf'.format(county), 'CURRENT')

'./Marin County.pdf'

'MAP TITLE'